In [2]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

2025-06-12 17:11:16.385435: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 17:11:16.394672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749728476.405860    8957 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749728476.409218    8957 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-12 17:11:16.420432: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
x_train = np.array([
    [180.4, 10.3],
    [190.4, 12.3],
    [200.3, 14.4],
    [210.4, 13.4],
    [220.4, 15.4],
    [230.3, 16.4],
    [240.3, 18.4],
    [250.4, 14.4]
],dtype=np.float32)
y_train = np.array([0, 1, 1, 1, 0, 1, 0, 1],dtype=np.float32)

In [4]:
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(x_train)
x_train_normal = norm_l(x_train)
print(x_train_normal)

tf.Tensor(
[[-1.5267953  -1.7532969 ]
 [-1.0901004  -0.8927832 ]
 [-0.6577722   0.01075585]
 [-0.21671087 -0.41950098]
 [ 0.21998394  0.44101268]
 [ 0.65231216  0.87126946]
 [ 1.089007    1.7317832 ]
 [ 1.5300683   0.01075585]], shape=(8, 2), dtype=float32)


2025-06-12 17:11:17.924584: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-06-12 17:11:17.924613: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: lenovoideapad
2025-06-12 17:11:17.924617: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: lenovoideapad
2025-06-12 17:11:17.924777: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.120.0
2025-06-12 17:11:17.924791: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.120.0
2025-06-12 17:11:17.924794: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.120.0


In [48]:
xTiled = np.tile(x_train_normal,(1000,1))
yTiled = np.tile(y_train.reshape(-1,1),(1000,1))
print(xTiled.shape)
print(yTiled.shape)

(8000, 2)
(8000, 1)


In [28]:
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3,activation='sigmoid',name='layer1'),
        Dense(1,activation='sigmoid',name='layer2')
    ]
)

In [30]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 3)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
w1,b1 = model.get_layer("layer1").get_weights()
w2,b2 = model.get_layer("layer2").get_weights()
print(f"w1 : {w1}")
print(f"b1 : {b1}")
print(f"w2 : {w2}")
print(f"b2 : {b2}")

w1 : [[-1.0808218   0.8839834   0.83698535]
 [-0.8502499   0.54768264 -0.02083206]]
b1 : [0. 0. 0.]
w2 : [[-0.00696421]
 [-1.0995295 ]
 [ 0.26036823]]
b2 : [0.]


In [50]:
model.compile(
    loss =  tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)
model.fit(
    xTiled,yTiled,            
    epochs=10,
)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 611us/step - loss: 0.6623
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 0.5742
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - loss: 0.5309
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 0.4738
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - loss: 0.4212
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - loss: 0.3812
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - loss: 0.3658
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - loss: 0.3450
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - loss: 0.3294
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - loss: 0.3333


In [52]:
w1,b1 = model.get_layer("layer1").get_weights()
w2,b2 = model.get_layer("layer2").get_weights()
print(f"w1 : {w1}")
print(f"b1 : {b1}")
print(f"w2 : {w2}")
print(f"b2 : {b2}")

w1 : [[ 1.3497322  -0.8072149   0.43966955]
 [-3.1675754   2.3279064   6.3427715 ]]
b1 : [ 3.1803777 -1.5770558  8.735964 ]
w2 : [[ 0.14028654]
 [-6.612513  ]
 [ 5.6374364 ]]
b2 : [-2.6712503]


In [60]:
x_test = np.array([[190.4,12.3],[180.4,10.3]])
x_val = norm_l(x_test)
predictions = model.predict(x_val)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


In [62]:
print(predictions)

[[0.910934  ]
 [0.08660088]]
